In [141]:
import pandas as pd
from numpy import array

In [142]:
data = pd.read_csv("https://raw.githubusercontent.com/dhamvi01/Univariate-Time-Series-using-LSTM/master/airline-passengers.csv")
data

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [143]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [144]:
# define univariate time series
passengers = data['Passengers'].values
# transform to a supervised learning problem
X, y = split_sequence(passengers, 2)
print(X.shape, y.shape)
dfX = pd.DataFrame(X, columns=["Xt-2", "Xt-1"])
dfy = pd.DataFrame(y, columns=["Xt"])

df = pd.concat((dfX, dfy), axis = 1)
df

(142, 2) (142,)


,Xt-2,Xt-1,Xt
0,112,118,132
1,118,132,129
2,132,129,121
3,129,121,135
4,121,135,148
...,...,...,...
137,535,622,606
138,622,606,508
139,606,508,461
140,508,461,390


# KNN

In [145]:
from sklearn.preprocessing import MinMaxScaler

#mengambil nama kolom
judul = df.columns.copy() 

#menghitung hasil normalisasi + menampilkan
scaler = MinMaxScaler()
scaler.fit(df)
X_normal = scaler.transform(df)
hasil = pd.DataFrame(X_normal,columns=judul)
hasil

,Xt-2,Xt-1,Xt
0,0.015444,0.027027,0.054054
1,0.027027,0.054054,0.048263
2,0.054054,0.048263,0.032819
3,0.048263,0.032819,0.059846
4,0.032819,0.059846,0.084942
...,...,...,...
137,0.832046,1.000000,0.969112
138,1.000000,0.969112,0.779923
139,0.969112,0.779923,0.689189
140,0.779923,0.689189,0.552124


In [146]:
hasil.describe()

,Xt-2,Xt-1,Xt
count,142.000000,142.000000,142.000000
mean,0.336791,0.340570,0.344839
std,0.231210,0.230304,0.230069
min,0.000000,0.000000,0.000000
25%,0.146718,0.147201,0.149614
50%,0.308880,0.311776,0.316602
75%,0.490830,0.493726,0.497104
max,1.000000,1.000000,1.000000


In [161]:
# ambil data
X = hasil.drop(columns="Xt")
y = hasil.Xt

In [148]:
# split data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)
print(X_train.shape)
print(X_test.shape)

(113, 2)
(29, 2)


In [157]:
# metode knn
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [158]:
y_pred = neigh.predict(X_test)

In [151]:
y_pred

array([0.06241956, 0.33011583, 0.23938224, 0.04697555, 0.07014157,
       0.36422136, 0.51930502, 0.48262548, 0.1016731 , 0.04890605,
       0.14736165, 0.46653797, 0.11068211, 0.17310167, 0.25353925,
       0.82110682, 0.35842986, 0.46203346, 0.14350064, 0.07464607,
       0.06306306, 0.82110682, 0.11969112, 0.42471042, 0.40862291,
       0.19047619, 0.61389961, 0.05984556, 0.23101673])

In [152]:
y_test

12     0.071429
64     0.382239
41     0.266409
11     0.042471
5      0.084942
96     0.486486
111    0.638996
118    0.494208
19     0.055985
2      0.032819
25     0.113900
88     0.596525
34     0.129344
29     0.183398
72     0.314672
134    0.710425
83     0.333977
120    0.583012
35     0.146718
16     0.127413
20     0.019305
101    0.700772
26     0.131274
85     0.403475
92     0.322394
47     0.177606
130    0.604247
24     0.142857
63     0.308880
Name: Xt, dtype: float64

In [159]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

0.004332341737480868